In [1]:
import random
import syft as sy
from syft.core import utils
from syft.core.frameworks.torch import utils as torch_utils
from syft.core.frameworks import encode

from syft.core.frameworks.torch.tensor import _GeneralizedPointerTensor
from syft.mpc import spdz
from syft.core.frameworks.torch.tensor import _MPCTensor
import torch
import torch.nn.functional as F
from torch.autograd import Variable as Var
import json

hook = sy.TorchHook(verbose=True)

me = hook.local_worker
me.is_client_worker = False

bob = sy.VirtualWorker(id="bob", hook=hook, is_client_worker=False)
alice = sy.VirtualWorker(id="alice", hook=hook, is_client_worker=False)
james = sy.VirtualWorker(id="james", hook=hook, is_client_worker=False)

x = torch.LongTensor([[1,2],[3,4]])
y = torch.LongTensor([[5,6],[7,8]])

In [2]:
xptr = x.share(bob, alice)
yptr = y.share(bob, alice)
z = xptr.mm(yptr)

In [4]:
z.get()


 18  21
 43  49
[syft.core.frameworks.torch.tensor.LongTensor of size 2x2]

In [5]:
x = torch.LongTensor([[1,2],[3,4]])
y = torch.LongTensor([[5,6],[7,8]])

In [6]:
x.mm(y)


 19  22
 43  50
[syft.core.frameworks.torch.tensor.LongTensor of size 2x2]